In [4]:
import random
import time
import sys
import os
import datetime

from sklearn.metrics import roc_auc_score, precision_recall_fscore_support, accuracy_score
import numpy as np
import tensorflow as tf

In [5]:
# 配置config
class TrainConfig(object):
    epochs = 15
    decay_rate = 0.92
    learning_rate = 0.01
    evaluate_every = 100
    checkpoint_every = 100
    max_grad_norm = 3.0


class ModelConfig(object):
    hidden_layers = [200]
    dropout_keep_prob = 0.6


class Config(object):
    batch_size = 10
    num_skills = 267
    input_size = num_skills * 2

    trainConfig = TrainConfig()
    modelConfig = ModelConfig()

In [6]:
# 实例化config
config = Config()

In [3]:
# 生成数据
class DataGenerator(object):
    # 导入的seqs是train_seqs，或者是test_seqs
    def __init__(self, fileName, config):
        self.fileName = fileName
        self.train_seqs = []
        self.test_seqs = []
        self.infer_seqs = []
        self.batch_size = config.batch_size
        self.pos = 0
        self.end = False
        self.num_skills = config.num_skills
        self.skills_to_int = {}  # 知识点到索引的映射
        self.int_to_skills = {}  # 索引到知识点的映射

    def read_file(self):
        # 从文件中读取数据，返回读取出来的数据和知识点个数
        # 保存每个学生的做题信息 {学生id: [[知识点id，答题结果], [知识点id，答题结果], ...]}，用一个二元列表来表示一个学生的答题信息
        seqs_by_student = {}
        skills = []  # 统计知识点的数量，之后输入的向量长度就是两倍的知识点数量
        count = 0
        with open(self.fileName, 'r') as f:
            for line in f:
                count += 1
                if count > 1:
                    fields = line.strip().split(",")  # 一个列表，[学生id，知识点id，答题结果]
                    student, skill, is_correct = int(fields[0]), int(fields[1]), int(fields[2])
                    skills.append(skill)  # skill实际上是用该题所属知识点来表示的
                    seqs_by_student[student] = seqs_by_student.get(student, []) + [[skill, is_correct]]  # 保存每个学生的做题信息
        return seqs_by_student, list(set(skills))

    def gen_dict(self, unique_skills):
        """
        构建知识点映射表，将知识点id映射到[0, 1, 2...]表示
        :param unique_skills: 无重复的知识点列表
        :return:
        """
        sorted_skills = sorted(unique_skills)
        skills_to_int = {}
        int_to_skills = {}
        for i in range(len(sorted_skills)):
            skills_to_int[sorted_skills[i]] = i
            int_to_skills[i] = sorted_skills[i]

        self.skills_to_int = skills_to_int
        self.int_to_skills = int_to_skills

    def split_dataset(self, seqs_by_student, sample_rate=0.2, random_seed=1):
        # 将数据分割成测试集和训练集
        sorted_keys = sorted(seqs_by_student.keys())  # 得到排好序的学生id的列表

        random.seed(random_seed)
        # 随机抽取学生id，将这部分学生作为测试集
        test_keys = set(random.sample(sorted_keys, int(len(sorted_keys) * sample_rate)))

        # 此时是一个三层的列表来表示的，最外层的列表中的每一个列表表示一个学生的做题信息
        test_seqs = [seqs_by_student[k] for k in seqs_by_student if k in test_keys]
        train_seqs = [seqs_by_student[k] for k in seqs_by_student if k not in test_keys]
        return train_seqs, test_seqs

    def gen_attr(self, is_infer=False):
        """
        生成待处理的数据集
        :param is_infer: 判断当前是训练模型还是利用模型进行预测
        :return:
        """
        if is_infer:
            seqs_by_students, skills = self.read_file()
            self.infer_seqs = seqs_by_students
        else:
            seqs_by_students, skills = self.read_file()
            train_seqs, test_seqs = self.split_dataset(seqs_by_students)
            self.train_seqs = train_seqs
            self.test_seqs = test_seqs

        self.gen_dict(skills)  # 生成知识点到索引的映射字典

    def pad_sequences(self, sequences, maxlen=None, value=0.):
        # 按每个batch中最长的序列进行补全, 传入的sequences是二层列表
        # 统计一个batch中每个序列的长度，其实等于seqs_len
        lengths = [len(s) for s in sequences]
        # 统计下该batch中序列的数量
        nb_samples = len(sequences)
        # 如果没有传入maxlen参数就自动获取最大的序列长度
        if maxlen is None:
            maxlen = np.max(lengths)
        # 构建x矩阵
        x = (np.ones((nb_samples, maxlen)) * value).astype(np.int32)

        # 遍历batch，去除每一个序列
        for idx, s in enumerate(sequences):
            trunc = np.asarray(s, dtype=np.int32)
            x[idx, :len(trunc)] = trunc

        return x

    def num_to_one_hot(self, num, dim):
        # 将题目转换成one-hot的形式， 其中dim=num_skills * 2，前半段表示错误，后半段表示正确
        base = np.zeros(dim)
        if num >= 0:
            base[num] += 1
        return base

    def format_data(self, seqs):
        # 生成输入数据和输出数据，输入数据是每条序列的前n-1个元素，输出数据是每条序列的后n-1个元素

        # 统计一个batch_size中每条序列的长度，在这里不对序列固定长度，通过条用tf.nn.dynamic_rnn让序列长度可以不固定
        seq_len = np.array(list(map(lambda seq: len(seq) - 1, seqs)))
        max_len = max(seq_len)  # 获得一个batch_size中最大的长度
        # i表示第i条数据，j只从0到len(i)-1，x作为输入只取前len(i)-1个，sequences=[j[0] + num_skills * j[1], ....]
        # 此时要将知识点id j[0] 转换成index表示
        x_sequences = np.array([[(self.skills_to_int[j[0]] + self.num_skills * j[1]) for j in i[:-1]] for i in seqs])
        # 将输入的序列用-1进行补全，补全后的长度为当前batch的最大序列长度
        x = self.pad_sequences(x_sequences, maxlen=max_len, value=-1)

        # 构建输入值input_x，x为一个二层列表，i表示一个学生的做题信息，也就是一个序列，j就是一道题的信息
        input_x = np.array([[self.num_to_one_hot(j, self.num_skills * 2) for j in i] for i in x])

        # 遍历batch_size，然后取每条序列的后len(i)-1 个元素中的知识点id为target_id
        target_id_seqs = np.array([[self.skills_to_int[j[0]] for j in i[1:]] for i in seqs])
        target_id = self.pad_sequences(target_id_seqs, maxlen=max_len, value=0)

        # 同target_id
        target_correctness_seqs = np.array([[j[1] for j in i[1:]] for i in seqs])
        target_correctness = self.pad_sequences(target_correctness_seqs, maxlen=max_len, value=0)

        return dict(input_x=input_x, target_id=target_id, target_correctness=target_correctness,
                    seq_len=seq_len, max_len=max_len)

    def next_batch(self, seqs):
        # 接收一个序列，生成batch

        length = len(seqs)
        num_batchs = length // self.batch_size
        start = 0
        for i in range(num_batchs):
            batch_seqs = seqs[start: start + self.batch_size]
            start += self.batch_size
            params = self.format_data(batch_seqs)

            yield params

In [12]:
fileName = "./data/knowledgeTracing.csv"
dataGen = DataGenerator(fileName, config)
dataGen.gen_attr()
train_seqs = dataGen.train_seqs
params = next(dataGen.next_batch(train_seqs))
print("train_seqs: {}".format(dataGen.train_seqs.shape))
print("test_seqs: {}".format(dataGen.test_seqs.shape))
print("input_x shape: {}".format(params['input_x'].shape))
print(params["input_x"][0][0])

input_x shape: (10, 64, 534)
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  

In [13]:
# 构建模型
class TensorFlowDKT(object):
    def __init__(self, config):
        # 导入配置好的参数
        self.hiddens = hiddens = config.modelConfig.hidden_layers
        self.num_skills = num_skills = config.num_skills
        self.input_size = input_size = config.input_size
        self.batch_size = batch_size = config.batch_size
        self.keep_prob_value = config.modelConfig.dropout_keep_prob

        # 定义需要喂给模型的参数
        self.max_steps = tf.placeholder(tf.int32, name="max_steps")  # 当前batch中最大序列长度
        self.input_data = tf.placeholder(tf.float32, [batch_size, None, input_size], name="input_x")

        self.sequence_len = tf.placeholder(tf.int32, [batch_size], name="sequence_len")
        self.keep_prob = tf.placeholder(tf.float32, name="keep_prob")  # dropout keep prob

        self.target_id = tf.placeholder(tf.int32, [batch_size, None], name="target_id")
        self.target_correctness = tf.placeholder(tf.float32, [batch_size, None], name="target_correctness")
        self.flat_target_correctness = None

        # 构建lstm模型结构
        hidden_layers = []
        for idx, hidden_size in enumerate(hiddens):
            lstm_layer = tf.nn.rnn_cell.LSTMCell(num_units=hidden_size, state_is_tuple=True)
            hidden_layer = tf.nn.rnn_cell.DropoutWrapper(cell=lstm_layer,
                                                         output_keep_prob=self.keep_prob)
            hidden_layers.append(hidden_layer)
        self.hidden_cell = tf.nn.rnn_cell.MultiRNNCell(cells=hidden_layers, state_is_tuple=True)

        # 采用动态rnn，动态输入序列的长度
        outputs, self.current_state = tf.nn.dynamic_rnn(cell=self.hidden_cell,
                                                        inputs=self.input_data,
                                                        sequence_length=self.sequence_len,
                                                        dtype=tf.float32)

        # 隐层到输出层的权重系数[最后隐层的神经元数量，知识点数]
        output_w = tf.get_variable("W", [hiddens[-1], num_skills])
        output_b = tf.get_variable("b", [num_skills])

        self.output = tf.reshape(outputs, [batch_size * self.max_steps, hiddens[-1]])
        # 因为权值共享的原因，对生成的矩阵[batch_size * self.max_steps, num_skills]中的每一行都加上b
        self.logits = tf.matmul(self.output, output_w) + output_b

        self.mat_logits = tf.reshape(self.logits, [batch_size, self.max_steps, num_skills])

        # 对每个batch中每个序列中的每个时间点的输出中的每个值进行sigmoid计算，这里的值表示对某个知识点的掌握情况，
        # 每个时间点都会输出对所有知识点的掌握情况
        self.pred_all = tf.sigmoid(self.mat_logits, name="pred_all")

        # 计算损失loss
        flat_logits = tf.reshape(self.logits, [-1])

        flat_target_correctness = tf.reshape(self.target_correctness, [-1])
        self.flat_target_correctness = flat_target_correctness

        flat_base_target_index = tf.range(batch_size * self.max_steps) * num_skills

        # 因为flat_logits的长度为batch_size * num_steps * num_skills，我们要根据每一步的target_id将其长度变成batch_size * num_steps
        flat_base_target_id = tf.reshape(self.target_id, [-1])

        flat_target_id = flat_base_target_id + flat_base_target_index
        # gather是从一个tensor中切片一个子集
        flat_target_logits = tf.gather(flat_logits, flat_target_id)

        # 对切片后的数据进行sigmoid转换
        self.pred = tf.sigmoid(tf.reshape(flat_target_logits, [batch_size, self.max_steps]), name="pred")
        # 将sigmoid后的值表示为0或1
        self.binary_pred = tf.cast(tf.greater_equal(self.pred, 0.5), tf.float32, name="binary_pred")

        # 定义损失函数
        with tf.name_scope("loss"):
            # flat_target_logits_sigmoid = tf.nn.log_softmax(flat_target_logits)
            # self.loss = -tf.reduce_mean(flat_target_correctness * flat_target_logits_sigmoid)
            self.loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=flat_target_correctness,
                                                                               logits=flat_target_logits))


In [15]:
# 训练模型
def mean(item):
    return sum(item) / len(item)


def gen_metrics(sequence_len, binary_pred, pred, target_correctness):
    """
    生成auc和accuracy的指标值
    :param sequence_len: 每一个batch中各序列的长度组成的列表
    :param binary_pred:
    :param pred:
    :param target_correctness:
    :return:
    """
    binary_preds = []
    preds = []
    target_correctnesses = []
    for seq_idx, seq_len in enumerate(sequence_len):
        binary_preds.append(binary_pred[seq_idx, :seq_len])
        preds.append(pred[seq_idx, :seq_len])
        target_correctnesses.append(target_correctness[seq_idx, :seq_len])

    new_binary_pred = np.concatenate(binary_preds)
    new_pred = np.concatenate(preds)
    new_target_correctness = np.concatenate(target_correctnesses)

    auc = roc_auc_score(new_target_correctness, new_pred)
    accuracy = accuracy_score(new_target_correctness, new_binary_pred)

    return auc, accuracy


class DKTEngine(object):

    def __init__(self):
        self.config = Config()
        self.train_dkt = None
        self.test_dkt = None
        self.sess = None
        self.global_step = 0

    def add_gradient_noise(self, grad, stddev=1e-3, name=None):
        """
        Adds gradient noise as described in http://arxiv.org/abs/1511.06807 [2].
        """
        with tf.op_scope([grad, stddev], name, "add_gradient_noise") as name:
            grad = tf.convert_to_tensor(grad, name="grad")
            gn = tf.random_normal(tf.shape(grad), stddev=stddev)
            return tf.add(grad, gn, name=name)

    def train_step(self, params, train_op, train_summary_op, train_summary_writer):
        """
        A single training step
        """
        dkt = self.train_dkt
        sess = self.sess
        global_step = self.global_step

        feed_dict = {dkt.input_data: params['input_x'],
                     dkt.target_id: params['target_id'],
                     dkt.target_correctness: params['target_correctness'],
                     dkt.max_steps: params['max_len'],
                     dkt.sequence_len: params['seq_len'],
                     dkt.keep_prob: self.config.modelConfig.dropout_keep_prob}

        _, step, summaries, loss, binary_pred, pred, target_correctness = sess.run(
            [train_op, global_step, train_summary_op, dkt.loss, dkt.binary_pred, dkt.pred, dkt.target_correctness],
            feed_dict)

        auc, accuracy = gen_metrics(params['seq_len'], binary_pred, pred, target_correctness)

        time_str = datetime.datetime.now().isoformat()
        print("train: {}: step {}, loss {}, acc {}, auc: {}".format(time_str, step, loss, accuracy, auc))
        train_summary_writer.add_summary(summaries, step)

    def dev_step(self, params, dev_summary_op, writer=None):
        """
        Evaluates model on a dev set
        """
        dkt = self.test_dkt
        sess = self.sess
        global_step = self.global_step

        feed_dict = {dkt.input_data: params['input_x'],
                     dkt.target_id: params['target_id'],
                     dkt.target_correctness: params['target_correctness'],
                     dkt.max_steps: params['max_len'],
                     dkt.sequence_len: params['seq_len'],
                     dkt.keep_prob: 1.0}
        step, summaries, loss, pred, binary_pred, target_correctness = sess.run(
            [global_step, dev_summary_op, dkt.loss, dkt.pred, dkt.binary_pred, dkt.target_correctness],
            feed_dict)

        auc, accuracy = gen_metrics(params['seq_len'], binary_pred, pred, target_correctness)
        # precision, recall, f_score = precision_recall_fscore_support(target_correctness, binary_pred)

        if writer:
            writer.add_summary(summaries, step)

        return loss, accuracy, auc

    def run_epoch(self, fileName):
        """
        训练模型
        :param filePath:
        :return:
        """

        # 实例化配置参数对象
        config = Config()

        # 实例化数据生成对象
        dataGen = DataGenerator(fileName, config)
        dataGen.gen_attr()  # 生成训练集和测试集

        train_seqs = dataGen.train_seqs
        test_seqs = dataGen.test_seqs

        session_conf = tf.ConfigProto(
            allow_soft_placement=True,
            log_device_placement=False
        )
        sess = tf.Session(config=session_conf)
        self.sess = sess

        with sess.as_default():
            # 实例化dkt模型对象
            with tf.name_scope("train"):
                with tf.variable_scope("dkt", reuse=None):
                    train_dkt = TensorFlowDKT(config)

            with tf.name_scope("test"):
                with tf.variable_scope("dkt", reuse=True):
                    test_dkt = TensorFlowDKT(config)

            self.train_dkt = train_dkt
            self.test_dkt = test_dkt

            global_step = tf.Variable(0, name="global_step", trainable=False)
            self.global_step = global_step

            # 定义一个优化器
            optimizer = tf.train.AdamOptimizer(config.trainConfig.learning_rate)
            grads_and_vars = optimizer.compute_gradients(train_dkt.loss)

            # 对梯度进行截断，并且加上梯度噪音
            grads_and_vars = [(tf.clip_by_norm(g, config.trainConfig.max_grad_norm), v)
                              for g, v in grads_and_vars if g is not None]
            # grads_and_vars = [(self.add_gradient_noise(g), v) for g, v in grads_and_vars]

            # 定义图中最后的节点
            train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step, name="train_op")

            # 保存各种变量或结果的值
            grad_summaries = []
            for g, v in grads_and_vars:
                if g is not None:
                    grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
                    sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                    grad_summaries.append(grad_hist_summary)
                    grad_summaries.append(sparsity_summary)
            grad_summaries_merged = tf.summary.merge(grad_summaries)

            timestamp = str(int(time.time()))
            out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
            print("writing to {}".format(out_dir))

            # 训练时的 Summaries
            train_loss_summary = tf.summary.scalar("loss", train_dkt.loss)
            train_summary_op = tf.summary.merge([train_loss_summary, grad_summaries_merged])
            train_summary_dir = os.path.join(out_dir, "summaries", "train")
            train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

            # 测试时的 summaries
            test_loss_summary = tf.summary.scalar("loss", test_dkt.loss)
            dev_summary_op = tf.summary.merge([test_loss_summary])
            dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
            dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

            saver = tf.train.Saver(tf.global_variables())

            sess.run(tf.global_variables_initializer())

            print("初始化完毕，开始训练")
            for i in range(config.trainConfig.epochs):
                np.random.shuffle(train_seqs)
                for params in dataGen.next_batch(train_seqs):
                    # 批次获得训练集，训练模型
                    self.train_step(params, train_op, train_summary_op, train_summary_writer)

                    current_step = tf.train.global_step(sess, global_step)
                    # train_step.run(feed_dict={x: batch_train[0], y_actual: batch_train[1], keep_prob: 0.5})
                    # 对结果进行记录
                    if current_step % config.trainConfig.evaluate_every == 0:
                        print("\nEvaluation:")
                        # 获得测试数据

                        losses = []
                        accuracys = []
                        aucs = []
                        for params in dataGen.next_batch(test_seqs):
                            loss, accuracy, auc = self.dev_step(params, dev_summary_op, writer=None)
                            losses.append(loss)
                            accuracys.append(accuracy)
                            aucs.append(auc)

                        time_str = datetime.datetime.now().isoformat()
                        print("dev: {}, step: {}, loss: {}, acc: {}, auc: {}".
                              format(time_str, current_step, mean(losses), mean(accuracys), mean(aucs)))

                    if current_step % config.trainConfig.checkpoint_every == 0:
                        path = saver.save(sess, "model/my-model", global_step=current_step)
                        print("Saved model checkpoint to {}\n".format(path))


if __name__ == "__main__":
    fileName = "./data/knowledgeTracing.csv"
    dktEngine = DKTEngine()
    dktEngine.run_epoch(fileName)


INFO:tensorflow:Summary name dkt/rnn/multi_rnn_cell/cell_0/lstm_cell/kernel:0/grad/hist is illegal; using dkt/rnn/multi_rnn_cell/cell_0/lstm_cell/kernel_0/grad/hist instead.
INFO:tensorflow:Summary name dkt/rnn/multi_rnn_cell/cell_0/lstm_cell/kernel:0/grad/sparsity is illegal; using dkt/rnn/multi_rnn_cell/cell_0/lstm_cell/kernel_0/grad/sparsity instead.
INFO:tensorflow:Summary name dkt/rnn/multi_rnn_cell/cell_0/lstm_cell/bias:0/grad/hist is illegal; using dkt/rnn/multi_rnn_cell/cell_0/lstm_cell/bias_0/grad/hist instead.
INFO:tensorflow:Summary name dkt/rnn/multi_rnn_cell/cell_0/lstm_cell/bias:0/grad/sparsity is illegal; using dkt/rnn/multi_rnn_cell/cell_0/lstm_cell/bias_0/grad/sparsity instead.
INFO:tensorflow:Summary name dkt/W:0/grad/hist is illegal; using dkt/W_0/grad/hist instead.
INFO:tensorflow:Summary name dkt/W:0/grad/sparsity is illegal; using dkt/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name dkt/b:0/grad/hist is illegal; using dkt/b_0/grad/hist instead.
INFO:tensorf

/home/jiangxinyang848/anaconda3/envs/jiang/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


writing to /data4T/share/jiangxinyang848/dkt/runs/1540468172
初始化完毕，开始训练
train: 2018-10-25T19:49:33.935153: step 1, loss 0.6947927474975586, acc 0.48619447779111646, auc: 0.4918510713031261
train: 2018-10-25T19:49:34.348814: step 2, loss 0.6826914548873901, acc 0.6371428571428571, auc: 0.6983210827963604
train: 2018-10-25T19:49:34.704649: step 3, loss 0.6839419603347778, acc 0.6334250343878954, auc: 0.6529206054989187
train: 2018-10-25T19:49:35.045240: step 4, loss 0.5792971253395081, acc 0.7582903463522476, auc: 0.7443520990050958
train: 2018-10-25T19:49:35.680104: step 5, loss 0.6496233344078064, acc 0.685807150595883, auc: 0.6368848386233709
train: 2018-10-25T19:49:36.113332: step 6, loss 0.7021452784538269, acc 0.6653771760154739, auc: 0.6905129335684891
train: 2018-10-25T19:49:36.550396: step 7, loss 0.6026129126548767, acc 0.7809239940387481, auc: 0.6576149582242394
train: 2018-10-25T19:49:37.134755: step 8, loss 0.6327204704284668, acc 0.7117493472584856, auc: 0.6644625131488314


train: 2018-10-25T19:50:03.061089: step 71, loss 0.44575297832489014, acc 0.7790575916230367, auc: 0.7659228432709096
train: 2018-10-25T19:50:03.586987: step 72, loss 0.4188094437122345, acc 0.8216258879242304, auc: 0.7884786377984263
train: 2018-10-25T19:50:03.942773: step 73, loss 0.42349565029144287, acc 0.8021555042340262, auc: 0.8061997440249298
train: 2018-10-25T19:50:04.428370: step 74, loss 0.41317886114120483, acc 0.8313253012048193, auc: 0.7348704914064391
train: 2018-10-25T19:50:04.631821: step 75, loss 0.4182058274745941, acc 0.8059701492537313, auc: 0.7992179487179487
train: 2018-10-25T19:50:05.144946: step 76, loss 0.47731468081474304, acc 0.7345739471106758, auc: 0.6866062077585791
train: 2018-10-25T19:50:05.563459: step 77, loss 0.42427000403404236, acc 0.7962447844228094, auc: 0.8876734366388952
train: 2018-10-25T19:50:06.163138: step 78, loss 0.5819935202598572, acc 0.5634813817855541, auc: 0.5255064451999883
train: 2018-10-25T19:50:06.872868: step 79, loss 0.44822278

train: 2018-10-25T19:50:33.648530: step 140, loss 0.31492897868156433, acc 0.8316755929609794, auc: 0.8659561322211926
train: 2018-10-25T19:50:33.966876: step 141, loss 0.3292032480239868, acc 0.8365724381625441, auc: 0.8041174336650083
train: 2018-10-25T19:50:34.450675: step 142, loss 0.33803704380989075, acc 0.796236559139785, auc: 0.8286320531629785
train: 2018-10-25T19:50:34.857635: step 143, loss 0.32796889543533325, acc 0.825287356321839, auc: 0.8828642831306341
train: 2018-10-25T19:50:35.340654: step 144, loss 0.317012220621109, acc 0.8214793741109531, auc: 0.8155734884458526
train: 2018-10-25T19:50:35.630550: step 145, loss 0.3441636264324188, acc 0.8243840808591283, auc: 0.8107985227050167
train: 2018-10-25T19:50:35.911809: step 146, loss 0.3596339821815491, acc 0.773394495412844, auc: 0.7932408229495147
train: 2018-10-25T19:50:36.368825: step 147, loss 0.35572630167007446, acc 0.759175788795879, auc: 0.8088736222775286
train: 2018-10-25T19:50:36.748172: step 148, loss 0.30913

...

train: 2018-10-25T19:53:00.212601: step 484, loss 0.2342788577079773, acc 0.8620689655172413, auc: 0.8789074406836382
train: 2018-10-25T19:53:00.565961: step 485, loss 0.20330120623111725, acc 0.8044280442804428, auc: 0.8374578984730172
train: 2018-10-25T19:53:01.030761: step 486, loss 0.16138909757137299, acc 0.8424, auc: 0.8880917348286488
train: 2018-10-25T19:53:01.310526: step 487, loss 0.1837901622056961, acc 0.8390446521287642, auc: 0.9060304699424542
train: 2018-10-25T19:53:01.827921: step 488, loss 0.23207269608974457, acc 0.7957941339236303, auc: 0.835271909531806
train: 2018-10-25T19:53:02.230025: step 489, loss 0.18221166729927063, acc 0.8396880415944541, auc: 0.8493218134522482
train: 2018-10-25T19:53:02.554461: step 490, loss 0.16567261517047882, acc 0.887719298245614, auc: 0.8879798868636077
train: 2018-10-25T19:53:02.736290: step 491, loss 0.17341169714927673, acc 0.8603351955307262, auc: 0.8881711812790511
train: 2018-10-25T19:53:03.181404: step 492, loss 0.200845882296

train: 2018-10-25T19:53:28.347574: step 552, loss 0.1839083433151245, acc 0.8631940469376074, auc: 0.9302833055521929
train: 2018-10-25T19:53:28.843109: step 553, loss 0.13273639976978302, acc 0.8768844221105527, auc: 0.8533134216912451
train: 2018-10-25T19:53:29.557541: step 554, loss 0.18769443035125732, acc 0.800383877159309, auc: 0.884826198826954
train: 2018-10-25T19:53:30.075305: step 555, loss 0.14059986174106598, acc 0.8707539353769677, auc: 0.8767720912401764
train: 2018-10-25T19:53:30.557814: step 556, loss 0.18032091856002808, acc 0.7922178988326848, auc: 0.8464426877470355
train: 2018-10-25T19:53:30.984436: step 557, loss 0.21715065836906433, acc 0.7998866213151927, auc: 0.8644112406867508
train: 2018-10-25T19:53:31.490231: step 558, loss 0.21283672749996185, acc 0.7814530419373893, auc: 0.849125161227072
train: 2018-10-25T19:53:31.769529: step 559, loss 0.17430315911769867, acc 0.857824427480916, auc: 0.8941339136163915
train: 2018-10-25T19:53:32.135229: step 560, loss 0.1

train: 2018-10-25T19:53:57.199739: step 620, loss 0.1731693297624588, acc 0.911620294599018, auc: 0.9183915862006438
train: 2018-10-25T19:53:57.501023: step 621, loss 0.18130500614643097, acc 0.8875614898102601, auc: 0.8745383057309662
train: 2018-10-25T19:53:57.958288: step 622, loss 0.17037518322467804, acc 0.8635265700483091, auc: 0.8868365024594376
train: 2018-10-25T19:53:58.508403: step 623, loss 0.1358027458190918, acc 0.8482084690553746, auc: 0.8819978814857613
train: 2018-10-25T19:53:58.788139: step 624, loss 0.17110413312911987, acc 0.8545454545454545, auc: 0.9176759620034349
train: 2018-10-25T19:53:59.182990: step 625, loss 0.12518854439258575, acc 0.8940092165898618, auc: 0.9204758254289713
train: 2018-10-25T19:53:59.538632: step 626, loss 0.19260793924331665, acc 0.8539119804400978, auc: 0.8628052020239176
train: 2018-10-25T19:53:59.972447: step 627, loss 0.1517626941204071, acc 0.8771498771498771, auc: 0.9399569892473119
train: 2018-10-25T19:54:00.194583: step 628, loss 0.

train: 2018-10-25T19:54:25.487807: step 690, loss 0.2039531022310257, acc 0.8913857677902621, auc: 0.9396717092642931
train: 2018-10-25T19:54:26.196847: step 691, loss 0.11873752623796463, acc 0.8509705698184096, auc: 0.9243834107079272
train: 2018-10-25T19:54:26.433168: step 692, loss 0.17917697131633759, acc 0.8704974271012007, auc: 0.8837047912454671
train: 2018-10-25T19:54:26.773420: step 693, loss 0.17377030849456787, acc 0.8628691983122363, auc: 0.8913764813126709
train: 2018-10-25T19:54:27.167873: step 694, loss 0.15853343904018402, acc 0.8768115942028986, auc: 0.9138911252425016
train: 2018-10-25T19:54:27.562693: step 695, loss 0.2330554872751236, acc 0.8029336734693877, auc: 0.8560185052974271
train: 2018-10-25T19:54:28.066648: step 696, loss 0.13150537014007568, acc 0.8372930165586753, auc: 0.8990731582225154
train: 2018-10-25T19:54:28.623703: step 697, loss 0.17580658197402954, acc 0.8180048661800486, auc: 0.8865204434474885
train: 2018-10-25T19:54:29.168945: step 698, loss 

train: 2018-10-25T19:54:55.982648: step 758, loss 0.13239185512065887, acc 0.9162264150943397, auc: 0.9266345334120271
train: 2018-10-25T19:54:56.312374: step 759, loss 0.15930967032909393, acc 0.8813920454545454, auc: 0.9374420316877504
train: 2018-10-25T19:54:56.796604: step 760, loss 0.11861255019903183, acc 0.8808888888888889, auc: 0.9157937147461724
train: 2018-10-25T19:54:57.113950: step 761, loss 0.14390060305595398, acc 0.9097496706192358, auc: 0.9222820105173046
train: 2018-10-25T19:54:57.504950: step 762, loss 0.1822786182165146, acc 0.810905892700088, auc: 0.890052019780361
train: 2018-10-25T19:54:58.068728: step 763, loss 0.1429971605539322, acc 0.8249084249084249, auc: 0.8804738793872092
train: 2018-10-25T19:54:58.513529: step 764, loss 0.15866243839263916, acc 0.8579846788450206, auc: 0.9047275110015711
train: 2018-10-25T19:54:59.039733: step 765, loss 0.15494102239608765, acc 0.8337547408343868, auc: 0.9071478074596773
train: 2018-10-25T19:54:59.351942: step 766, loss 0.

In [16]:
# 模型预测
def load_model(fileName):
    # 实例化配置参数对象
    config = Config()

    # 实例化数据生成对象
    dataGen = DataGenerator(fileName, config)
    dataGen.gen_attr()  # 生成训练集和测试集

    test_seqs = dataGen.test_seqs

    with tf.Session() as sess:

        accuracys = []
        aucs = []
        step = 1

        for params in dataGen.next_batch(test_seqs):
            print("step: {}".format(step))

            saver = tf.train.import_meta_graph("model/my-model-800.meta")
            saver.restore(sess, tf.train.latest_checkpoint("model/"))

            # 获得默认的计算图结构
            graph = tf.get_default_graph()

            # 获得需要喂给模型的参数，输出的结果依赖的输入值
            input_x = graph.get_operation_by_name("test/dkt/input_x").outputs[0]
            target_id = graph.get_operation_by_name("test/dkt/target_id").outputs[0]
            keep_prob = graph.get_operation_by_name("test/dkt/keep_prob").outputs[0]
            max_steps = graph.get_operation_by_name("test/dkt/max_steps").outputs[0]
            sequence_len = graph.get_operation_by_name("test/dkt/sequence_len").outputs[0]

            # 获得输出的结果
            pred_all = graph.get_tensor_by_name("test/dkt/pred_all:0")
            pred = graph.get_tensor_by_name("test/dkt/pred:0")
            binary_pred = graph.get_tensor_by_name("test/dkt/binary_pred:0")

            target_correctness = params['target_correctness']
            pred_all, pred, binary_pred = sess.run([pred_all, pred, binary_pred],
                                                   feed_dict={input_x: params["input_x"],
                                                              target_id: params["target_id"],
                                                              keep_prob: 1.0,
                                                              max_steps: params["max_len"],
                                                              sequence_len: params["seq_len"]})

            auc, acc = gen_metrics(params["seq_len"], binary_pred, pred, target_correctness)
            print(auc, acc)
            accuracys.append(acc)
            aucs.append(auc)
            step += 1

        aucMean = mean(aucs)
        accMean = mean(accuracys)

        print("inference  auc: {}  acc: {}".format(aucMean, accMean))


if __name__ == "__main__":
    fileName = "./data/knowledgeTracing.csv"
    load_model(fileName)

step: 1
INFO:tensorflow:Restoring parameters from model/my-model-800
0.651437901683 0.630281690141
step: 2
INFO:tensorflow:Restoring parameters from model/my-model-800
0.852083988143 0.892568659128
step: 3
INFO:tensorflow:Restoring parameters from model/my-model-800
0.821706453103 0.947848761408
step: 4
INFO:tensorflow:Restoring parameters from model/my-model-800
0.761215150662 0.814510097233
step: 5
INFO:tensorflow:Restoring parameters from model/my-model-800
0.714353598734 0.834129692833
step: 6
INFO:tensorflow:Restoring parameters from model/my-model-800
0.828215200101 0.784359093012
step: 7
INFO:tensorflow:Restoring parameters from model/my-model-800
0.835155906546 0.773483365949
step: 8
INFO:tensorflow:Restoring parameters from model/my-model-800
0.811198237741 0.750379362671
step: 9
INFO:tensorflow:Restoring parameters from model/my-model-800
0.875720174433 0.840540540541
step: 10
INFO:tensorflow:Restoring parameters from model/my-model-800
0.828990439151 0.830357142857
step: 11
